In [1]:
! pip install geopy

     -------------------------------------- 119.8/119.8 kB 7.3 MB/s eta 0:00:00
     ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00


In [1]:
import pandas as pd
from geopy.geocoders import Nominatim

In [2]:
df=pd.read_csv('data/CoffeeBean.csv')
df.address

0                              서울시 강남구 학동로 211 1층  
1                        서울시 강남구 광평로 280 수서동 724호  
2                         서울시 강남구 논현로 566 강남차병원1층  
3                             서울시 서초구 강남대로 369 1층  
4                        서울 강남구 삼성동 159 코엑스몰 지하2층  
                           ...                     
238    인천광역시 중구 운서동 2868 인천공항 제2터미널 1층 연번(23-124)  
239                         부산광역시 부산진구 서전로 12, 1층  
240                     경기도 성남시 분당구 야탑로64 DS타워 1층  
241                                서울시 용산구 한강대로95  
242                     서울특별시 강남구 선릉로 433 세방빌딩 1층  
Name: address, Length: 243, dtype: object

In [3]:

geolocoder = Nominatim(user_agent='South Korea')
a= ['서울특별시 강남구 학동로 211', '서울특별시 강남구 광평로 280', '서울특별시 강남구 논현로 556']
for i, loc in enumerate(a):
    geo=geolocoder.geocode(loc)
    crd=(geo.latitude, geo.longitude)
    print(geo, crd)
    

학동로, 삼성동, 삼성1동, 강남구, 서울, 06078, 대한민국 (37.5199698, 127.056947)
중앙물산, 280, 광평로, 수서동, 강남구, 서울, 06367, 대한민국 (37.487311, 127.1030943)
논현로, 신사동, 압구정동, 신사동, 강남구, 서울, 06025, 대한민국 (37.5272651, 127.0283095)
